In [1]:
import os
import math

import numba
import numpy as np
import atomium
import Bio
from Bio import Seq
import sys
sys.path.append('/nfs/rmadaj/graph_toolbox/feature')
import torch as th
import pandas as pd
from scipy.spatial import distance_matrix
# protein_letters_3to1 = Bio.SeqUtils.IUPACData.protein_letters_3to1_extended
# protein_letters_3to1 = {k.upper() : v for k,v in protein_letters_3to1.items()}

In [2]:
from feature import calc_named

In [3]:
import sys
from feature import calc_named
sys.path.append('/nfs/rmadaj/graph_toolbox/feature')
calc_named(chain = None, pdb_loc = '../kfra/af2/r751/hth-motif.pdb', t = 9)


,disulfide,hydrophobic,cation_pi,arg_arg,salt_bridge,hbond,c,lj,e,cbcb,dist,ca_vs_cb,self,is_seq,is_seq_not,is_struct,res1,res2
0,0.0,0.0,1.0,0.0,0.0,0.0,-3.610619,10.000000,10.000000,0.999993,0.000000,0.0,1.0,0.0,0.0,0.0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,-3.610619,10.000000,10.000000,0.999211,0.184365,0.0,0.0,1.0,0.0,0.0,0,1
2,0.0,0.0,1.0,0.0,0.0,0.0,-3.610619,10.000000,10.000000,0.999477,0.114666,1.0,0.0,0.0,1.0,1.0,0,6
3,0.0,0.0,1.0,0.0,0.0,0.0,-3.610619,10.000000,10.000000,1.000334,0.198436,0.0,0.0,0.0,1.0,1.0,0,11
4,0.0,0.0,1.0,0.0,0.0,0.0,-3.610619,10.000000,10.000000,0.990571,0.188867,0.0,0.0,0.0,1.0,1.0,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,0.0,0.0,0.0,0.0,0.0,0.0,1.498504,-0.011733,1.486771,0.999993,0.000000,0.0,1.0,0.0,0.0,0.0,55,55
651,0.0,0.0,0.0,0.0,0.0,0.0,1.075542,0.000000,1.075542,1.003220,0.148980,0.0,0.0,0.0,1.0,1.0,56,8
652,0.0,0.0,0.0,0.0,0.0,0.0,1.075542,0.000000,1.075542,1.002484,0.260343,0.0,0.0,0.0,1.0,1.0,56,43
653,0.0,0.0,0.0,0.0,0.0,0.0,1.075542,0.000000,1.075542,0.990104,0.281460,0.0,0.0,0.0,1.0,1.0,56,44


In [4]:
type(calc_named(chain = None, pdb_loc = '../kfra/af2/r751/hth-motif.pdb', t = 9))

pandas.core.frame.DataFrame

In [ ]:
@numba.njit(parallel=True)
def numba_jit_scalar_distance_parallel(xyz):
    rows = xyz.shape[0]
    output = np.empty((rows, rows), dtype=np.float32)
    for i in numba.prange(rows):
        cols = rows - i
        for j in numba.prange(cols):
            tmp = 0.0
            tmp += (xyz[i, 0] - xyz[j, 0])**2
            tmp += (xyz[i, 1] - xyz[j, 1])**2
            tmp += (xyz[i, 2] - xyz[j, 2])**2
            tmp = math.sqrt(tmp) 
            output[i,j] = tmp
            output[j,i] = tmp
    return output

In [ ]:
def get_atom_xyz(atoms, atom_name):
    for a in atoms:
        if a.name == atom_name:
            return a.location
    return (np.nan, np.nan, np.nan)

def get_ss_label(residue):
    '''
    E, H or C label from atomium
    '''
    if residue.helix:
        return 'H'
    elif residue.strand:
        return 'E'
    else:
        return 'C'

In [ ]:
def parse_graph_data_numba(path_pdb, chain):
    
    if not os.path.isfile(path_pdb):
        FileNotFoundError('no such file', path_pdb)
    file = atomium.open(path_pdb)
    chain = file.model.chain(chain)
    preparation_dict = dict()
    for i, r in enumerate(chain.residues()):
        r_atoms = r.atoms()
        preparation_dict[i] = {'aa' : protein_letters_3to1[r.name],
                                    'charge' : r.charge,
                                    'CA' : get_atom_xyz(r_atoms, 'CA'),
                                    'CB' : get_atom_xyz(r_atoms, 'CB'),
                                    'ss_label' : get_ss_label(r)
                                   }

        ca_xyz = np.asarray(list(map(lambda v : v['CA'], preparation_dict.values())), dtype=np.float32)
        sequence = list(map(lambda v : v['aa'], preparation_dict.values()))
        ca_ca_matrix = numba_jit_scalar_distance_parallel(ca_xyz)
    return ca_ca_matrix, sequence

In [ ]:
def parse_graph_data_torch(path_pdb, chain):
    
    if not os.path.isfile(path_pdb):
        FileNotFoundError('no such file', path_pdb)
    file = atomium.open(path_pdb)
    chain = file.model.chain(chain)
    preparation_dict = dict()
    for i, r in enumerate(chain.residues()):
        r_atoms = r.atoms()
        preparation_dict[i] = {'aa' : protein_letters_3to1[r.name],
                                    'charge' : r.charge,
                                    'CA' : get_atom_xyz(r_atoms, 'CA'),
                                    'CB' : get_atom_xyz(r_atoms, 'CB'),
                                    'ss_label' : get_ss_label(r)
                                   }

        ca_xyz = th.FloatTensor(list(map(lambda v : v['CA'], preparation_dict.values())))
        sequence = list(map(lambda v : v['aa'], preparation_dict.values()))

        ca_ca_matrix = th.cdist(ca_xyz, ca_xyz)
    return ca_ca_matrix, sequence

In [ ]:
def parse_graph_data(path_pdb, chain):
    
    if not os.path.isfile(path_pdb):
        FileNotFoundError('no such file', path_pdb)
    file = atomium.open(path_pdb)
    chain = file.model.chain(chain)
    preparation_dict = dict()
    for i, r in enumerate(chain.residues()):
        r_atoms = r.atoms()
        preparation_dict[i] = {'aa' : protein_letters_3to1[r.name],
                                    'charge' : r.charge,
                                    'CA' : get_atom_xyz(r_atoms, 'CA'),
                                    'CB' : get_atom_xyz(r_atoms, 'CB'),
                                    'ss_label' : get_ss_label(r)
                                   }

        ca_xyz = np.asarray(list(map(lambda v : v['CA'], preparation_dict.values())), dtype=np.float32)
        sequence = list(map(lambda v : v['aa'], preparation_dict.values()))

        ca_ca_matrix = distance_matrix(ca_xyz, ca_xyz)
    return ca_ca_matrix, sequence

In [ ]:
path = '../kfra/md/hth_attached/map/contactmap_mapped.pdb'
chain = 'A'

In [ ]:
%timeit parse_graph_data(path, chain)

In [ ]:
%timeit parse_graph_data_torch(path, chain)

In [ ]:
%timeit parse_graph_data_numba(path, chain)

In [ ]:
a,b = parse_graph_data_torch(path, chain)

In [ ]:
a.element_size()*a.nelement() / 1024